<a href="https://colab.research.google.com/github/robbybrodie/time_as_computation_cost/blob/main/notebooks/02_Tension_Bandgaps_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tension Bandgaps Experiment

This notebook explores micro fitting and model selection with synthetic data.

## Theory

- **DoF Law**: DoF(N) = exp(-a*(1-N)) + noise
- **Mapping**: ψ = DoF^β
- **Model Comparison**: Exponential vs Polynomial vs Power Law
- **Selection Criteria**: AIC, BIC, Cross-validation

## Setup

In [ ]:
# Colab bootstrap
REPO_URL = "https://github.com/robbybrodie/time_as_computation_cost.git"
REPO_NAME = "time_as_computation_cost"

import pathlib
if not pathlib.Path(REPO_NAME).exists():
    !git clone $REPO_URL
%cd $REPO_NAME

if pathlib.Path("pyproject.toml").exists():
    !pip install -e .

In [ ]:
from experiments.run_tension_bandgaps import main
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys

# Ensure we can import the modules
repo_root = Path().resolve()
sys.path.insert(0, str(repo_root / "src"))

from tacc.bandgaps.tension import run_demo, run_experiment

## Basic Experiment

In [ ]:
# Run the basic experiment
results = main()

print("Model fitting completed!")
print(f"a_hat: {results['fitted_params']['a_hat']:.3f} (true: 2.0)")
print(f"beta_hat: {results['fitted_params']['beta_hat']:.3f} (true: 1.5)")

## Interactive Demo

In [ ]:
# Create and display demo visualization
fig = run_demo(n_points=50, noise_sigma=0.05, a_true=2.0, beta_true=1.5)
plt.show()

## Noise Level Exploration

In [ ]:
# Explore different noise levels
noise_levels = [0.01, 0.05, 0.1, 0.2]
a_true, beta_true = 2.0, 1.5

results_by_noise = []
for noise in noise_levels:
    exp_result = run_experiment(noise_sigma=noise, a_true=a_true, beta_true=beta_true)
    results_by_noise.append(exp_result)

# Plot parameter recovery
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

a_hats = [r['fitted_params']['a_hat'] for r in results_by_noise]
beta_hats = [r['fitted_params']['beta_hat'] for r in results_by_noise]

ax1.plot(noise_levels, a_hats, 'bo-', linewidth=2, markersize=8, label='Fitted a')
ax1.axhline(y=a_true, color='r', linestyle='--', label=f'True a = {a_true}')
ax1.set_xlabel('Noise Level σ')
ax1.set_ylabel('Parameter a')
ax1.set_title('Parameter Recovery: a')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(noise_levels, beta_hats, 'go-', linewidth=2, markersize=8, label='Fitted β')
ax2.axhline(y=beta_true, color='r', linestyle='--', label=f'True β = {beta_true}')
ax2.set_xlabel('Noise Level σ')
ax2.set_ylabel('Parameter β')
ax2.set_title('Parameter Recovery: β')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Parameter Recovery Summary:")
for i, noise in enumerate(noise_levels):
    print(f"σ={noise}: a_hat={a_hats[i]:.3f}, β_hat={beta_hats[i]:.3f}")

## Model Comparison Analysis

In [ ]:
# Detailed model comparison
noise_sigma = 0.05
detailed_results = run_experiment(noise_sigma=noise_sigma)

# Extract model comparison data
models = ['exponential', 'polynomial', 'power_law']
model_names = ['Exponential', 'Polynomial-2', 'Power Law']

aic_scores = [detailed_results['model_comparison'][model]['aic'] for model in models]
bic_scores = [detailed_results['model_comparison'][model]['bic'] for model in models]
cv_scores = [detailed_results['model_comparison'][model]['cv_score'] for model in models]
cv_stds = [detailed_results['model_comparison'][model]['cv_std'] for model in models]

# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# AIC/BIC comparison
x_pos = np.arange(len(model_names))
width = 0.35

ax1.bar(x_pos - width/2, aic_scores, width, label='AIC', alpha=0.8)
ax1.bar(x_pos + width/2, bic_scores, width, label='BIC', alpha=0.8)
ax1.set_xlabel('Model')
ax1.set_ylabel('Score (lower is better)')
ax1.set_title('Information Criteria')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(model_names, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Cross-validation comparison
ax2.bar(x_pos, cv_scores, yerr=cv_stds, capsize=5, alpha=0.8)
ax2.set_xlabel('Model')
ax2.set_ylabel('CV MSE (lower is better)')
ax2.set_title('Cross-Validation Performance')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(model_names, rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find best model
best_aic_idx = np.argmin(aic_scores)
best_cv_idx = np.argmin(cv_scores)

print(f"\nModel Comparison Results (σ = {noise_sigma}):")
print("=" * 40)
for i, (model, name) in enumerate(zip(models, model_names)):
    print(f"{name}:")
    print(f"  AIC: {aic_scores[i]:.2f}")
    print(f"  BIC: {bic_scores[i]:.2f}")
    print(f"  CV MSE: {cv_scores[i]:.4f} ± {cv_stds[i]:.4f}")
    print()

print(f"Best model (AIC): {model_names[best_aic_idx]}")
print(f"Best model (CV): {model_names[best_cv_idx]}")

## Key Insights

1. **Parameter Recovery**: Exponential model successfully recovers true parameters
2. **Noise Robustness**: Performance degrades gracefully with increasing noise
3. **Model Selection**: AIC/BIC correctly identify exponential as best model
4. **Cross-validation**: Confirms model ranking from information criteria

## Physical Interpretation

- **DoF(N)**: "Degrees of freedom" as function of computational capacity
- **Exponential Law**: Suggests exponential suppression as capacity decreases
- **β Parameter**: Controls nonlinear mapping to observable quantity ψ

**Note**: This uses synthetic data - real physics validation needed!